In [6]:
import json
import tabulate

# engine_kind = ['sm', 'sm-base', 'sm-opt', 'v8', 'v8-liftoff', 'v8-turbofan', 'jsc', 'jsc-int','jsc-bbq','jsc-omg', 'wizeng','wizeng-int','wizeng-jit','wizeng-dyn','wasmtime','wasmer','wasmer-base']
engine_kind = ['sm', 'v8', 'jsc', 'wasmtime','wasmer', 'wizeng-int','wizeng-jit','wizeng-dyn']

with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ1: Applicability')

print('RQ1-1: Accuracy Experiment')
rq1_results = [['Test Name', 'Trace Match']] + sorted([[testname, 'o' if metrics[testname]['trace_match']['wasabi'] else ''] for testname in metrics]) + [['Total', sum([1 for testname in metrics if metrics[testname]['trace_match']['wasabi']])]]
print(tabulate.tabulate(rq1_results, tablefmt="latex"))

print('RQ1-2: Portability Experiment')
def get_engine_runtime(testname):
    return [metrics[testname]['replay_metrics'][engine]['benchmark'].get('runtime') for engine in engine_kind]
rq22_results = [['Test name', *engine_kind]] + sorted([[testname, *get_engine_runtime(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi'] ])
print(tabulate.tabulate(rq22_results, tablefmt="latex"))

RQ1: Applicability
RQ1-1: Accuracy Experiment
\begin{tabular}{ll}
\hline
 Test Name       & Trace Match \\
 boa             & o           \\
 bullet          & o           \\
 commanderkeen   & o           \\
 factorial       & o           \\
 ffmpeg          & o           \\
 fib             & o           \\
 figma-startpage & o           \\
 fractals        &             \\
 funky-kart      & o           \\
 game-of-life    & o           \\
 gotemplate      &             \\
 guiicons        & o           \\
 hnset-bench     &             \\
 hydro           & o           \\
 image-convolute &             \\
 jqkungfu        & o           \\
 jsc             & o           \\
 lichess         &             \\
 livesplit       &             \\
 mandelbrot      & o           \\
 multiplyDouble  & o           \\
 multiplyInt     & o           \\
 ogv             & o           \\
 onnxjs          &             \\
 pacalc          & o           \\
 parquet         & o           \\
 pathfind

KeyError: 'sm'

In [10]:
import json
import tabulate, scipy.stats

with open('metrics.json', 'r') as f: metrics = json.load(f)
del metrics['funky-kart'] 
print('RQ2: Performance')

print('RQ2-1: Performance Overhead')

def get_runtime_slowdown(testname):
    result = []
    for i in range(5):
        original_runtime = metrics[testname]['record_metrics']['original'][i]['mean'] * metrics[testname]['record_metrics']['original'][i]['samples']
        record_runtime = metrics[testname]['record_metrics']['instrumented'][i]['mean'] * metrics[testname]['record_metrics']['instrumented'][i]['samples']
        result.append(record_runtime/original_runtime)
    return result
def get_cycles_slowdown(testname):
    result = []
    for i in range(5):
        original_cycles = sum(metrics[testname]['record_metrics']['original'][i]['cycles'] if metrics[testname]['record_metrics']['original'][i]['cycles'] != -1 else [-1])
        record_cycles = sum(metrics[testname]['record_metrics']['instrumented'][i]['cycles'] if metrics[testname]['record_metrics']['instrumented'][i]['cycles'] != -1 else [-1])
        result.append(record_cycles/original_cycles)
    return result
runtime_slowdown = [['Test Name', 'Runtime Slowdown[1]', 'Runtime Slowdown[2]', 'Runtime Slowdown[3]','Runtime Slowdown[4]','Runtime Slowdown[5]']] + sorted([[testname, *get_runtime_slowdown(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi']])
gmean_row = ['Geometric Mean']
for i in range(1, len(runtime_slowdown[0])):
    column_values = [row[i] for row in runtime_slowdown[1:]]
    print(column_values)
    column_gmean = scipy.stats.gmean(column_values)
    gmean_row.append(column_gmean)
runtime_slowdown.append(gmean_row)
print(tabulate.tabulate(runtime_slowdown, tablefmt="latex"))

cycles_slowdown = [['Test Name', 'Cycles Slowdown[1]', 'Cycles Slowdown[2]', 'Cycles Slowdown[3]','Cycles Slowdown[4]','Cycles Slowdown[5]']] + sorted([[testname, *get_cycles_slowdown(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi']])
gmean_row = ['Geometric Mean']
for i in range(1, len(cycles_slowdown[0])):
    column_values = [row[i] for row in cycles_slowdown[1:]]
    column_gmean = scipy.stats.gmean(column_values)
    gmean_row.append(column_gmean)
cycles_slowdown.append(gmean_row)
print(tabulate.tabulate(cycles_slowdown, tablefmt="latex"))


print('RQ2-2: Performance Characteristics')
def get_metric(testname, metric):
    return metrics[testname]['replay_metrics']['wizeng-int']['custom'].get(metric)
rq22_results = [['Test name', 'replay proportion', 'pregen time', 'load time', 'validate time', 'spc time', 'start time', 'main time']] + [[testname, 'TODO', get_metric(testname, 'pregen:time_us'), get_metric(testname, 'load:time_us'), get_metric(testname, 'validate:time_us'), get_metric(testname, 'spc:time_us'), get_metric(testname, 'start:time_us'), get_metric(testname, 'main:time_us')] for testname in metrics if metrics[testname]['trace_match']['wasabi'] ]

print(tabulate.tabulate(rq22_results, tablefmt="latex"))

RQ2: Performance
RQ2-1: Performance Overhead
[1.0374262536873156, 25.024859395741768, 66.5258665013477, 0.8180997887742755, 1.0072760273050476, 4.8030778973492225, 0.5972367550234496, 0.9119613660806576, 21.23528060442619, 0.17086814631713662, 99.06626921832968, 0.8661218036218036, 14.509292681812822, 1.028668592021706, 1.0023799517516678, 0.5207313369373747, 1.625331910727839, 0.48844856033604567, 5.945180694553879, 3.168814590379557, 16.023357521650112, 38.971207502094806, 7.220030783271845, 0.6474824369892462, 0.8031347047133444, 14.636164065873524, 0.9802664945364009, 1.222429568417867, 1.3252893835269313]
[1.042859679447458, 25.231188128761083, 60.540631503760686, 1.061225631192074, 0.8951661695152668, 4.749771240222365, 0.6159212574024444, 1.3955223880597016, 20.16941682120438, 0.1604834415033319, 101.90023393030683, 0.9392634392634392, 14.586857580491163, 0.9961497893606736, 0.9986043778463348, 0.48281024887708696, 1.2861695347355004, 0.43649415137953607, 52.13295317996111, 3.60

KeyError: 'wizeng-int'

In [4]:
import json
import tabulate, scipy.stats

with open('metrics.json', 'r') as f: metrics = json.load(f)
del metrics['funky-kart']
print('RQ3: Effectiveness of Trace Reduction')

def get_runtime_slowdown(testname):
    result = []
    for i in range(5):
        disable_shadow_runtime = metrics[testname]['record_metrics']['disable_shadow'][i]['mean']
        record_runtime = metrics[testname]['record_metrics']['instrumented'][i]['mean']
        result.append(record_runtime/disable_shadow_runtime)
    return result
def get_cycles_slowdown(testname):
    result = []
    for i in range(5):
        disable_shadow_cycles = sum(metrics[testname]['record_metrics']['disable_shadow'][i]['cycles'] if metrics[testname]['record_metrics']['disable_shadow'][i]['cycles'] != -1 else [-1])
        record_cycles = sum(metrics[testname]['record_metrics']['instrumented'][i]['cycles'] if metrics[testname]['record_metrics']['instrumented'][i]['cycles'] != -1 else [-1])
        result.append(record_cycles/disable_shadow_cycles)
    return result

runtime_slowdown = [['Test Name', 'Runtime Slowdown[1]', 'Runtime Slowdown[2]', 'Runtime Slowdown[3]','Runtime Slowdown[4]','Runtime Slowdown[5]']] + sorted([[testname, *get_runtime_slowdown(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi']])
gmean_row = ['Geometric Mean']
for i in range(1, len(runtime_slowdown[0])):
    column_values = [row[i] for row in runtime_slowdown[1:]]
    column_gmean = scipy.stats.gmean(column_values)
    gmean_row.append(column_gmean)
runtime_slowdown.append(gmean_row)
print(tabulate.tabulate(runtime_slowdown, tablefmt="latex"))

# cycles_slowdown = [['Test Name', 'Cycles Slowdown[1]', 'Cycles Slowdown[2]', 'Cycles Slowdown[3]','Cycles Slowdown[4]','Cycles Slowdown[5]']] + sorted([[testname, *get_cycles_slowdown(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi']])
# gmean_row = ['Geometric Mean']
# for i in range(1, len(cycles_slowdown[0])):
#     column_values = [row[i] for row in cycles_slowdown[1:]]
#     column_gmean = scipy.stats.gmean(column_values)
#     gmean_row.append(column_gmean)
# cycles_slowdown.append(gmean_row)
# print(tabulate.tabulate(cycles_slowdown, tablefmt="latex"))

RQ3: Effectiveness of Trace Reduction
\begin{tabular}{llllll}
\hline
 Test Name       & Runtime Slowdown[1] & Runtime Slowdown[2] & Runtime Slowdown[3] & Runtime Slowdown[4] & Runtime Slowdown[5] \\
 boa             & 0.9691698243196694  & 1.0262096774193548  & 1.0297102786491974  & 1.0140692640692641  & 0.9939024390243902  \\
 bullet          & 1.0208500669344043  & 0.8957757855582924  & 0.872668586559462   & 1.1074238050839416  & 0.8569599309402202  \\
 commanderkeen   & 1.0134410404236776  & 0.9978491119439465  & 1.008451264375435   & 1.005978332672744   & 1.0089879502205834  \\
 factorial       & 0.8170818505338078  & 0.8287581699346405  & 0.9569721115537848  & 1.1162420382165605  & 0.9601837672281778  \\
 ffmpeg          & 1.2082494969818913  & 1.026634382566586   & 0.9771428571428572  & 1.0965811965811967  & 1.0426008968609866  \\
 fib             & 1.095077370084795   & 1.0299681884031688  & 1.0482184390644094  & 0.964738921765635   & 1.0033877946345116  \\
 figma-startpage & 1.

In [ ]:
import json
import tabulate
with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ4: Effectiveness of Replay Optimization')

def get_metric(testname, opt, time):
    metric = metrics[testname]['replay_metrics']['wizeng-int'][opt]
    if len(metric) == 0:
        return 0
    else:
        return metric[time]

print('RQ4-1: Load time')
time = 'load:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics if metrics[testname]['trace_match']['wasabi']]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))

print('RQ4-2: Validate time')
time = 'validate:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics if metrics[testname]['trace_match']['wasabi']]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))

print('RQ4-3: Main time')
time = 'main:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics if metrics[testname]['trace_match']['wasabi']]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))